In [14]:
import os
import json
from dotenv import load_dotenv
load_dotenv()

with open('../data/extracted_books/1 - Dune - Frank Herbert.json', 'r') as f:
    data = json.load(f)


In [28]:
from pprint import pprint as pp

def extract_content(data, result=None):
    if result is None:
        result = []
    for item in data:
        result.append(item['content'])
        if 'children' in item:
            extract_content(item['children'], result)
    return result

text_part = extract_content(data['data'])[10]

sub_parts = text_part.split('\n\n\n')
for sub_part in sub_parts:
    print(len(sub_part))

496
9793
443
3776


In [40]:
from openai import OpenAI
client = OpenAI(
    base_url='https://api.groq.com/openai/v1',
    api_key=os.environ.get("GROQ_API_KEY"),
)

summary_prompt = "You will be given a part of a book chapter between the <TEXT_PART> and </TEXT_PART> tags, you will summarize the actions that occur in that text. You will only output the summary, nothing else, keep it brief and factual, don't add any external knowledge, here is the text part: \n <TEXT_PART> {text_part} </TEXT_PART> \n remember, brief, concise, factual"

entity_extraction_prompt = "You will be given a part of a book chapter between the <TEXT_PART> and </TEXT_PART> tags. You will extract every named entity in that text and classify them as PERSON, LOCATION, ORGANIZATION or CONCEPT, don't add any external knowledge. You will output your answer in the following format : \n <entity> : <category> \n for example : Jean : PERSON \n The great wall : LOCATION \n Mireille : PERSON \n Here is the text part: \n <TEXT_PART> {text_part} </TEXT_PART>"

completion = client.chat.completions.create(
  model="llama3-8b-8192",
  messages=[
    {"role": "user", "content": entity_extraction_prompt.format(text_part=sub_parts[1])}
  ]
)

print(sub_parts[1])
print()
print()
print(completion.choices[0].message.content)

« E h bien, Jessica, qu'as-tu à dire pour toi-même ? » demanda la Révérende Mère.

Ce même jour, Paul avait subi l'épreuve et maintenant le crépuscule venait. Les deux femmes étaient seules dans le salon de Jessica ; Paul attendait dans la Chambre de Méditation d'où il ne pouvait percevoir la moindre parole.

Jessica se tenait devant les fenêtres ouvertes sur le sud. Elle voyait et ne voyait pas les couleurs qui s'étaient rassemblées sur la prairie, sur la rivière, avec le soir. Elle entendait et n'entendait pas les mots que prononçait la Révérende Mère. Elle avait déjà subi l'épreuve, tant d'années auparavant. Elle n'était alors qu'une fillette frêle aux cheveux couleur de bronze, au corps secoué par les tempêtes de la puberté. Une fillette qui avait subi l'examen de la Révérende Mère Gaius Helen Mohiam, Rectrice supérieure de l'école Bene Gesserit de Wallach IX. Aujourd'hui, elle contemplait sa main droite, pliait ses doigts et se souvenait de la souffrance, de la peur, de la colère.